In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
plt.rcParams['font.family']

## テキストから単語共起ネットワーク

In [ ]:
sentences = [
    '天皇は、日本国の象徴であり日本国民統合の象徴であって、この地位は、主権の存する日本国民の総意に基く。',
    '皇位は、世襲のものであって、国会の議決した皇室典範の定めるところにより、これを継承する。',
    '天皇の国事に関するすべての行為には、内閣の助言と承認を必要とし、内閣が、その責任を負う。',
    '天皇は、この憲法の定める国事に関する行為のみを行い、国政に関する権能を有しない。',
    '天皇は、法律の定めるところにより、その国事に関する行為を委任することができる。',
    '皇室典範の定めるところにより摂政を置くときは、摂政は、天皇の名でその国事に関する行為を行う。',
    'この場合には、前条第一項の規定を準用する。',
    '天皇は、国会の指名に基いて、内閣総理大臣を任命する。',
    '天皇は、内閣の指名に基いて、最高裁判所の長たる裁判官を任命する。',
    '天皇は、内閣の助言と承認により、国民のために、左の国事に関する行為を行う。',
    '憲法改正、法律、政令及び条約を公布すること。',
    '国会を召集すること。',
    '衆議院を解散すること。',
    '国会議員の総選挙の施行を公示すること。',
    '国務大臣及び法律の定めるその他の官吏の任免並びに全権委任状及び大使及び公使の信任状を認証すること。',
    '大赦、特赦、減刑、刑の執行の免除及び復権を認証すること。',
    '栄典を授与すること。',
    '批准書及び法律の定めるその他の外交文書を認証すること。',
    '外国の大使及び公使を接受すること。',
    '儀式を行うこと。',
    '皇室に財産を譲り渡し、又は皇室が、財産を譲り受け、若しくは賜与することは、国会の議決に基かなければならない。',
]

In [ ]:
### 形態素解析 janome
from janome.tokenizer import Tokenizer
janome = Tokenizer()
def ma_sentence(sentence):
    return [{'surface': x.surface, 'pos': x.part_of_speech, 'base_form': x.base_form} for x in janome.tokenize(sentence)]

ma_sentence('菓子を買ってもお金は減らず。')

In [ ]:
### 形態素解析 MeCab
import MeCab
mecab = MeCab.Tagger('-Ochasen')
def ma_sentence(sentence):
    mors = [x.split('\t') for x in mecab.parse(sentence).split('\n')]
    return [{'surface': mor[0], 'pos': mor[3], 'base_form': mor[2]} for mor in mors if len(mor) > 2]

ma_sentence('菓子を買ってもお金は減らず。')

In [ ]:
mors_list = [ma_sentence(sentence) for sentence in sentences]
print(mors_list[0][0:5])

In [ ]:
# ワードのリストを作成
import re
words_list = [{mor['surface'] for mor in mors if re.match('^名詞(?!.*(代名詞|接尾|非自立))', mor['pos'])} for mors in mors_list]
print(words_list[0])
# ワードの頻度
import collections
all_words = [x for row in words_list  for x in row]
ct_word = collections.Counter(all_words)
print(ct_word.most_common()[:5])

In [ ]:
# ワードペアのリストを作成
import itertools
all_pairs = [tuple(sorted(x)) for row in [list(itertools.combinations(words, 2)) for words in words_list] for x in row]
print(all_pairs[:5])
# ワードペアの頻度
ct_pair = collections.Counter(all_pairs)
print(ct_pair.most_common()[:5])

In [ ]:
# グラフ表示に使うワードを限定する
target_words = {x[0] for x in ct_word.most_common() if x[1] >= 1}
print(list(target_words)[:5])

In [ ]:
# node と edge の　DataFrame を作る
df_nodes = pd.DataFrame([
    {'label': i[0], 'count': i[1]}
    for i in ct_word.most_common() if i[0] in target_words
])
df_edges = pd.DataFrame([
    {'node1': i[0][0], 'node2': i[0][1], 'count': i[1]}
    for i in ct_pair.most_common() if i[0][0] in target_words and i[0][1] in target_words
])

In [ ]:
# 頻度 (count) から weight を計算する
# edge の　 weight は Jaccard 係数
import math
df_nodes['weight'] = df_nodes.apply(lambda x: math.log(x['count']), axis=1)
df_edges['weight'] = df_edges.apply(lambda x: x['count'] / (ct_word[x['node1']] + ct_word[x['node2']] - x['count']), axis=1)

In [ ]:
display(df_nodes.head(3), df_edges.head(3))

## dictデータからネットワーク

## グラフの描画

In [ ]:
G = nx.Graph()
#G = nx.from_pandas_edgelist(df_edges, 'node1', 'node2', 'weight')
G.add_edges_from(df_edges.apply(lambda x: (x['node1'], x['node2'], {'weight': x['weight']}), axis=1))
G.add_nodes_from(df_nodes.apply(lambda x: (x['label'], {'weight': x['weight']}), axis=1))

In [ ]:
pos = nx.spring_layout(G, k=0.6, seed=0)
width_list =[d['weight'] * 5 for (u, v, d) in G.edges(data=True)]
node_size_list = [d['weight'] * 1000 + 500 for (n, d) in G.nodes(data=True)]

fig, ax = plt.subplots(1,1, figsize=(15,10))

nx.draw_networkx_nodes(G, pos, node_size=node_size_list, linewidths=2,
                       # node_color=list(pr.values()), cmap=plt.cm.Reds,
                       node_color='pink', alpha=0.5, ax=ax)
nx.draw_networkx_edges(G, pos, width=width_list, edge_color='gray', alpha=0.1, ax=ax)
nx.draw_networkx_labels(G, pos, font_size=12, font_color="black", alpha=0.9, font_family="IPAGothic", ax=ax)

plt.show()